## Processing neural datasets
Process neural datasets into python structures for smaller storage size and faster access

In [1]:
%load_ext autoreload
%autoreload 2


import neural_datasets

from pprint import pprint

import os
if not os.path.exists('../data'):
    os.makedirs('../data')
if not os.path.exists('../data/datasets'):
    os.makedirs('../data/datasets')


datadir = '/scratches/ramanujan_2/dl543/'
datadir2 = '/scratches/sagarmatha_2/ktj21/data/crcns/'

We load the dataset for one session from the original files downloaded from [CRCNR.org](https://crcns.org/), which is then preprocessed and put into a pickle file in the ```../data/datasets``` directory.


    if mouse_id == 'Mouse12':
        """
        Anterior Thalamus:
        Electrode groups: 1-8.
        8 shank probe, Neuronexus Buz64 design. Shank #1 is the most lateral, #8 the most medial
        Probe is perpendicular to the midline (coronal plane), tilted by 15 degrees (tips pointed toward midline)
        , mounted on a movable drive.
        Insertion coordinates: AP -0.7mm, ML 0.5mm
        Approx depth from surface: 2.97mm

        medial Prefrontal (Prelimbic):
        Electrode Groups: 9-12
        4 tetrodes
        Insertion coordinates: AP 1.78 - 1.94mm, ML 0.5mm, depth: 1.7mm

        Hippocampus:
        Electrode group: 14.
        5 tungsten wires (fifty micrometer in diamater), one (at least) is in the pyramidal layer of CA1 as shown
         by the presence of Sharp-Wave/Ripples.
        Insertion coordinates: AP -2.2mm, ML 1.3mm, depth: 1.7mm
        """

        channels = {
            'ANT': [1, 2, 3, 4, 5, 6, 7, 8],
            'mPFC': [9, 10, 11, 12, 13], 
            'CA1': [14]
        }
    elif mouse_id == 'Mouse28':
        """
        Postsubiculum:
        Electrode groups 1-7.
        6 shank probe, Neuronexus Buz64sp design. Shank #1 is the most medial, #6 the most lateral, group #7 made
         of the four sites located above 4th shank (see design).
        Approx depth from surface: 1.28mm

        Anterior thalamus:
        Electrode groups 8-11.
        4 shank probe, Neuronexus Buz32 design. Shank #8 is the most lateral, #11 the most medial.
        Approx depth from surface: 2.56mm
        """
        channels = {
            'PoS': [1, 2, 3, 4, 5, 6, 7], 
            'ANT': [8, 9, 10, 11]
        }

    elif mouse_id == 'Mouse24':
        """
        Post-subiculum:
        Electrode Groups: 1-4
        4 shank probe, Neuronexus Buz32 design. Shank #1 is the most medial, #4 the most lateral
        Probe is perpendicular to the midline (coronal plane), tilted by 10 degrees (tips pointed away from midli
        ne), mounted on a movable drive.
        Insertion coordinates: AP -4.25mm, ML 1.2-1.9mm
        Approx depth from surface: 1.06mm

        Anterior Thalamus:
        Electrode groups: 5-8.
        4 shank probe, Neuronexus Buz32 design. Shank #1 (5th electrode group) is the most lateral, #4 (8th elect
        rode group) the most medial
        Probe is perpendicular to the midline (coronal plane), mounted on a movable drive.
        Insertion coordinates: AP -0.7mm, ML 0.5-1.2mm
        Approx depth from surface: 3.11mm
        """
        channels = {
            'PoS': [1, 2, 3, 4],
            'ANT': [5, 6, 7, 8]
        }

    elif mouse_id == 'Mouse25':
        """
        Post-subiculum:
        Electrode Groups: 1-4
        4 shank probe, Neuronexus Buz32 design. Shank #1 is the most medial, #4 the most lateral
        Probe is perpendicular to the midline (coronal plane), tilted by 10 degrees (tips pointed away from midli
        ne), mounted on a movable drive.
        Insertion coordinates: AP -4.25mm, ML 1.2-1.9mm
        Approx depth from surface: 2.4mm

        Anterior Thalamus:
        Electrode groups: 5-8.
        4 shank probe, Neuronexus Buz32 design. Shank #1 (5th electrode group) is the most lateral, #4 (8th elect
        rode group) the most medial
        Probe is perpendicular to the midline (coronal plane), mounted on a movable drive.
        Insertion coordinates: AP -0.7mm, ML 0.5-1.2mm
        Approx depth from surface: 2.72mm
        """
        channels = {
            'PoS': [1, 2, 3, 4],
            'ANT': [5, 6, 7, 8]
        }

In [2]:
mice_channels = {
    'Mouse12': {'ANT': [1, 2, 3, 4, 5, 6, 7, 8],
                'mPFC': [9, 10, 11, 12], 
                'CA1': [13]},
    'Mouse17': {'ANT': [1, 2, 3, 4, 5, 6, 7, 8],
                'CA1': [9]},
    'Mouse20': {'ANT': [1, 3, 4, 5, 6, 7, 8]},
    'Mouse24': {'PoS': [1, 2, 3, 4],
                'ANT': [5, 6, 7, 8]},
    'Mouse25': {'PoS': [1, 2, 3, 4],
                'ANT': [5, 6, 7, 8]},
    'Mouse28': {'PoS': [1, 2, 3, 4, 5, 6, 7], 
                'ANT': [8, 9, 10, 11]},
}

mice_sessions = {
    'Mouse12': ['120806', '120807', '120809', '120810' ], # '120808' is missing position files
    'Mouse17': ['130125', '130128', '130129', '130130', '130131', '130201', '130202', '130203', '130204'],
    'Mouse20': ['130514', '130515', '130516', '130517', '130520'],
    'Mouse24': ['131213', '131216', '131217','131218'],
    'Mouse25': ['140123', '140124', '140128', '140129', '140130', '140131', '140203', '140204', '140205', '140206'],
    'Mouse28': ['140310', '140311', '140312', '140313', '140317', '140318'],
} 

In [3]:
# Mice sleep/wake exploration, head direction cells
phase = 'wake'

for mouse_id in mice_sessions.keys():
    for session_id in mice_sessions[mouse_id]:
        print(mouse_id, session_id)
        
        channels = mice_channels[mouse_id]
        data_class = neural_datasets.peyrache_th1(datadir2+'/th-1/data/', mouse_id, session_id, channels)
        
        periods = data_class.get_periods()
        time_limits = [periods['wake'][0]['start'], periods['wake'][0]['end']]  # pick wake session
        savef = '/scratches/ramanujan_2/dl543/HDC_PartIII/th1_{}_{}_{}.p'.format(mouse_id, session_id, phase)
        data_class.load_preprocess_save(savef, time_limits)

Mouse12 120806
Behaviour time bin size 2.56e-02 s.
Empty channel shank 4 cluster 3.
{'HD': [{'index': 0, 'length': 1793}, {'index': 170752, 'length': 6145}, {'index': 349953, 'length': 7167}, {'index': 381697, 'length': 5119}, {'index': 397568, 'length': 14849}, {'index': 509696, 'length': 10241}, {'index': 640257, 'length': 5119}, {'index': 648961, 'length': 8192}, {'index': 1178880, 'length': 11264}, {'index': 1531137, 'length': 9216}, {'index': 1544449, 'length': 7167}, {'index': 1695489, 'length': 6143}, {'index': 1941249, 'length': 5632}, {'index': 2321665, 'length': 15871}, {'index': 2973952, 'length': 4608}, {'index': 3009280, 'length': 18432}, {'index': 3072257, 'length': 17408}, {'index': 4122369, 'length': 27647}, {'index': 4151040, 'length': 43008}, {'index': 4211456, 'length': 5121}, {'index': 5187329, 'length': 15360}, {'index': 5518081, 'length': 25600}, {'index': 5555457, 'length': 9215}, {'index': 5576449, 'length': 10752}, {'index': 5590785, 'length': 14848}, {'index':

/var/home/dl543/Documents/Learning/UCM-interpret/preprocess/neural_datasets.py:269: RuntimeWarning: invalid value encountered in long_scalars
  refract_viol[u] = (ISI[u] <= viol_ISI).sum()/len(ISI[u])


{'HD': [{'index': 0, 'length': 1793}, {'index': 48897, 'length': 5120}, {'index': 539393, 'length': 5119}, {'index': 828161, 'length': 5631}, {'index': 846080, 'length': 3072}, {'index': 1107712, 'length': 14336}, {'index': 1921793, 'length': 5631}, {'index': 2061057, 'length': 5631}, {'index': 3397889, 'length': 7680}, {'index': 4161793, 'length': 7680}, {'index': 6107392, 'length': 12801}, {'index': 6142721, 'length': 7680}, {'index': 6488832, 'length': 6145}, {'index': 6579969, 'length': 16383}, {'index': 7534849, 'length': 14336}, {'index': 7609600, 'length': 8705}, {'index': 7644928, 'length': 16897}, {'index': 7671040, 'length': 4097}, {'index': 7755520, 'length': 10241}, {'index': 7766273, 'length': 10752}, {'index': 8154368, 'length': 13825}, {'index': 8398081, 'length': 10240}, {'index': 8742657, 'length': 7168}, {'index': 8750337, 'length': 13824}, {'index': 9157376, 'length': 9728}, {'index': 9174784, 'length': 8193}, {'index': 10133760, 'length': 4609}, {'index': 10219777, 

In [ ]:
pprint(periods, indent=2, sort_dicts=False)

time_limits = [periods['wake'][0]['start'], periods['wake'][0]['end']]  # pick wake session